In [1]:
# First, import the relevant modules
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

The inital data set was provided on Kaggle collected by jtrofe (https://www.kaggle.com/jtrofe/beer-recipes) and using data from www.Brewersfriend.com. The initial data provided in this data set was a robust starting point for this project. However, additional data was acquired to supplement this inital set of 73,800+ entries. The original data came in two .csv files. The first (recipeData.csv) contains most of the information on the homebrews. The second (styleData.csv) contains the assignment for the styles of beer found in the recipeData file.

The data acqusitition for my project is in two parts. The first part is scraping ratings data from the website using the Beautiful Soup package. The second part is to use the API to obtain recipe data (e.g. ingredients, hops, yeast, etc.) for each entry.

A few separate files used in this notebook have been augmented from the original data set. The first (recipeData_urls_all.csv) only contains all of the urls in the original recipeData.csv document. These entries specifically are the subdirectories pertaining to each beer in the dataset. The second (recipe_id.csv) is generated below in part two and contains a list of each of the recipe id harvested from the urls in the original recipeData.csv document. 

1) Data Acquisition of Ratings Data via Beautiful Soup

In [15]:
# Establish headers, the base_url (or domain), and a list to accept data from Beautiful Soup

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36'
}

base_url = 'https://www.brewersfriend.com'
data = [["url", "rating", "reviews", "calories", "carbs"]]

In [16]:
# Define a function to help sift through the soup: ladel

def ladle(url):
    # This function takes the url given and requests the rating, review, calories, and carbs of the beer in question.
    # The entry is appended to the defined list.
    
    beer_html = requests.get(url, headers=headers).text

    soup = BeautifulSoup(beer_html, 'html5lib')

    rating = soup.find('span', {'itemprop': 'ratingValue'}) if soup.find('span', {'itemprop': 'ratingValue'}) != None else "NaN"
    review = soup.find('span', {'itemprop': 'reviewCount'}) if soup.find('span', {'itemprop': 'reviewCount'}) != None else "NaN"
    calories = soup.find('strong', {'class': 'calories'}) if soup.find('strong', {'class': 'calories'}) != None else "NaN"
    carbs = soup.find('strong', {'class': 'carbs'}) if soup.find('strong', {'class': 'carbs'}) != None else "NaN"

    temp = [url, 
            rating.text if rating != "NaN" else 'NaN', 
            review.text if review != "NaN" else 'NaN', 
            calories.text if calories != "NaN" else 'NaN', 
            carbs.text if carbs != "NaN" else 'NaN']
    data.append(temp)
        
    return data

In [17]:
# A dataframe of the full url is generated for each url fragment in Kaggle data set.

url_df = pd.read_csv('recipeData_urls_all.csv', header= None, engine = 'python', encoding = 'ISO-8859-1').apply(lambda x: base_url + x)

print(url_df.tail(10))

                                                       0
73851  https://www.brewersfriend.com/homebrew/recipe/...
73852  https://www.brewersfriend.com/homebrew/recipe/...
73853  https://www.brewersfriend.com/homebrew/recipe/...
73854  https://www.brewersfriend.com/homebrew/recipe/...
73855  https://www.brewersfriend.com/homebrew/recipe/...
73856  https://www.brewersfriend.com/homebrew/recipe/...
73857  https://www.brewersfriend.com/homebrew/recipe/...
73858  https://www.brewersfriend.com/homebrew/recipe/...
73859  https://www.brewersfriend.com/homebrew/recipe/...
73860  https://www.brewersfriend.com/homebrew/recipe/...


In [ ]:
# Write a for loop: for each full url in the dataframe "url_df", perform the ladel function. This will build up a list: data
# included a counter to show the status of scraping 

counter = 0
for index, row in url_df.iterrows():
    ladle(row[0])
    counter += 1
    if counter % 500 == 0:
        print (counter)
    else:
        continue

In [6]:
# Convert the list "data" into a dataframe: df
# This step took about 7 hours to complete and was done in a separate Jupyter notebook with the same code.

df = pd.DataFrame(data)
print(df.tail())

                                                      0    1    2  \
3433  https://www.brewersfriend.com/homebrew/recipe/...  NaN  NaN   
3434  https://www.brewersfriend.com/homebrew/recipe/...  NaN  NaN   
3435  https://www.brewersfriend.com/homebrew/recipe/...  NaN  NaN   
3436  https://www.brewersfriend.com/homebrew/recipe/...  NaN  NaN   
3437  https://www.brewersfriend.com/homebrew/recipe/...  NaN  NaN   

                 3       4  
3433           NaN     NaN  
3434  159 calories  17.2 g  
3435  153 calories  14.8 g  
3436  206 calories  22.4 g  
3437  435 calories  29.3 g  


In [9]:
# Save the webscraped data as a csv file for later: reviewData_all.csv

df.to_csv('reviewData_all.csv')

From step one, we have available review data from most of the entries. This was limited by two factors. 

The first limitation was from the data itself. Only entries that have reviews and ratings at the time of scraping will be included.

The second limitation is the time required for this scraping to occur. Due to how large the data set is, I experienced problems
scraping due to TimeOut Errors with the server. This was possibly due to over-requesting and being rejected from the server end to 
avoid a crash. To remedy this, the web scraping occured overnight and I was able to get all the review data.


2) Data Acquisition of Ingredient Data via Brewer's Friend API

First, the recipe ID numbers need to be extracted from the recipeData.csv original dataset. 

In [18]:
url_df = pd.read_csv('recipeData_urls.csv', header= None, engine = 'python', encoding = 'ISO-8859-1')

print(url_df.tail(10))

                                                       0
73851          /homebrew/recipe/view/615556/blonde-stout
73852        /homebrew/recipe/view/618629/session-simcoe
73853  /homebrew/recipe/view/602248/chris-ford-wheat-ipa
73854  /homebrew/recipe/view/603016/x-files-american-ale
73855           /homebrew/recipe/view/607368/unicorn-pee
73856         /homebrew/recipe/view/609673/amber-alfie-2
73857               /homebrew/recipe/view/610955/rye-ipa
73858                      /homebrew/recipe/view/586891/
73859                      /homebrew/recipe/view/603788/
73860  /homebrew/recipe/view/613776/elvis-juice-ipa-c...


In [3]:
new_list = []
for index,row in url_df.iterrows():
    temp = row[0].rsplit('/')
    new_list.append(temp)
    
rec_df = pd.DataFrame(new_list)
print(rec_df.tail(10))

      0         1       2     3       4                      5
73851    homebrew  recipe  view  615556           blonde-stout
73852    homebrew  recipe  view  618629         session-simcoe
73853    homebrew  recipe  view  602248   chris-ford-wheat-ipa
73854    homebrew  recipe  view  603016   x-files-american-ale
73855    homebrew  recipe  view  607368            unicorn-pee
73856    homebrew  recipe  view  609673          amber-alfie-2
73857    homebrew  recipe  view  610955                rye-ipa
73858    homebrew  recipe  view  586891                       
73859    homebrew  recipe  view  603788                       
73860    homebrew  recipe  view  613776  elvis-juice-ipa-clone


In [4]:
# Getting only the column with the recipe ID's, 

rec_id = rec_df[[4]]
print(rec_id.head())

       4
0   1633
1  16367
2   5920
3   5916
4  89534


In [5]:
# The list of recipe_IDs is saved for future reference: 'recipe_id.csv'
rec_id.to_csv('recipe_id.csv')

Using the newly created list of recipe IDs, rec_id, the ingredient list API can now be set up.

In [6]:
# Establish new headers and base url (domain) for the recipe and ingredients

my_headers = {'X-API-KEY': '1062c1a3650672bb65e9dc8c71bd7dfe4061166f'}
base_rec_url = 'https://api.brewersfriend.com/v1/recipes/'

In [7]:
# Create five empty dataframes, one for each category of ingredient and place in a dictionary: ingredient_dict

fermentables = pd.DataFrame()
hops = pd.DataFrame()
misc = pd.DataFrame()
mash = pd.DataFrame()
yeast = pd.DataFrame()

ingredient_dict = {"FERMENTABLE": fermentables, "HOP": hops, "MISC": misc, 'MASH': mash,'YEAST': yeast}

In [8]:
# Define a function to help parse through xml data for each recipe: xml_sift()

def xml_sift(xml_file, recipe_id, xpath_loc, df):
    
    """ This function looks at the xml data from Brewer's Friend recipe API output, specifically looking at the Fermentables, Hops, 
    Misc, Mash Steps, and Yeast used and adds the value of Recipe ID in a new column. The MASH portion of the if-else statement is 
    needed because the xpath is unique compared to the other ingredients. The try-except clause is required to ignore recipes that 
    do not contain a given ingredient type (usually MISC values are missing).""" 
    
    if 'MASH' in xpath_loc:
        try:
            temp = pd.read_xml(xml_file, xpath = "/RECIPES/RECIPE/MASH/MASH_STEPS/MASH_STEP")
            temp.insert(0, "Recipe_ID", recipe_id, True)
            return temp
        except ValueError: 
            pass 
    else:
        try:
            temp = pd.read_xml(xml_file, xpath = "/RECIPES/RECIPE/" + xpath_loc + 'S/' + xpath_loc)
            temp.insert(0, "Recipe_ID", recipe_id, True)
            return temp
        except ValueError: 
            pass

In [9]:
# Define a function to get ingredient data for a given recipe: gather_ingredients()

def gather_ingredients(recipe_id):
    
    """This function uses the domain (base_rec_url) and the input recipe_id to request an xml using the API from
    Brewer's Friend (https://docs.brewersfriend.com/api/recipes). This function uses standard request.get to retrieve
    the xml file. This file cannot be converted into a json. The for loop works through each of the five ingredient
    types using the dictionary ingredient_dict. The results from xml_sift are concatenated onto the respective dataframe
    by adding rows under the previous recipe (axis = 0). 
    
    The subfunction '.reset_index()' was required to get this to work because otherwise we were rewriting index 0-5 multiple times.
    The try-except clause is used to acknowledge missing entries or xpaths passed from the previous function (e.g.
    entries that don't have MISC ingredients listed will be ignored)."""
    
    url = base_rec_url + str(recipe_id) + '.xml'

    r = requests.get(url, headers=my_headers)

    xml_file = r.text
    
    for ingredient in ingredient_dict:
        try: 
            ingredient_dict[ingredient] = pd.concat([ingredient_dict[ingredient], xml_sift(xml_file, recipe_id, ingredient, ingredient_dict[ingredient])], axis = 0, ignore_index = True, sort = False).reset_index(drop=True)
        except:
            continue
    

In [10]:
# Write a for loop: for each recipe ID in the dataframe "rec_id", perform the gather_ingredients function. 

for index, row in rec_id.iterrows():
    gather_ingredients(row[4])   

In [11]:
# Checking the shape of the FERMENTABLE dataframe 

print(ingredient_dict['FERMENTABLE'].shape)

(294096, 10)


In [12]:
# Save each of the dataframes in 'ingredient_dict' as a csv file

for k,v in ingredient_dict.items():
    ingredient_dict[k].to_csv(path_or_buf = k+'.csv', index=False)

From step two, we have five CSV files that each contain all of the ingredient data for each recipe saved (e.g. 'FERMENTABLE.CSV'). 

In the data wrangling component, using the six CSV files gathered here and the original data set, we'll go through and 
design the final data set that will be used for modeling. 


3) Analyzing Data that was Gathered: (Review Data)

For the first portion of the data cleaning, 

In [2]:
ferm_df = pd.read_csv('FERMENTABLE.csv')
print(ferm_df.head())

   Recipe_ID                   NAME  VERSION     TYPE    AMOUNT  YIELD  COLOR  \
0       1633             Pale 2-Row        1    Grain  2.381360  80.43    1.8   
1       1633            White Wheat        1    Grain  0.907185  86.96    2.8   
2       1633             Pale 6-Row        1    Grain  0.907185  76.09    1.8   
3       1633            Flaked Corn        1  Adjunct  0.226796  86.96    0.5   
4       1633  Caramel / Crystal 20L        1    Grain  0.226796  76.09   20.0   

   ADD_AFTER_BOIL         ORIGIN  DIASTATIC_POWER  
0           False  United States              0.0  
1           False  United States              0.0  
2           False  United States              0.0  
3           False            NaN              0.0  
4           False  United States              0.0  


In [3]:
unique_ferm = ferm_df['NAME'].unique()
print(unique_ferm)

['Pale 2-Row' 'White Wheat' 'Pale 6-Row' ... 'Dry Malt Extract - Pale Ale'
 'Best Malz Rye' 'JWM Traditional Ale Malt']


In [4]:
hops_df = pd.read_csv('HOP.csv')
print(hops_df.head())

   Recipe_ID      NAME  VERSION  ALPHA    AMOUNT   USE USER_HOP_USE  TIME  \
0       1633   Cascade        1    6.2  0.014175  Boil         Boil  60.0   
1       1633   Cascade        1    6.2  0.014175  Boil         Boil  20.0   
2       1633      saaz        1    3.0  0.014175  Boil         Boil   5.0   
3      16367    Magnum        1   15.0  0.028350  Boil         Boil  50.0   
4      16367  Sterling        1    8.7  0.028350  Boil         Boil  10.0   

     FORM  HOP_TEMP  UTILIZATION  TEMPERATURE  
0  Pellet       NaN          NaN          NaN  
1  Pellet       NaN          NaN          NaN  
2  Pellet       NaN          NaN          NaN  
3  Pellet       NaN          NaN          NaN  
4  Pellet       NaN          NaN          NaN  


In [5]:
unique_hops_df = hops_df['NAME'].unique()
unique_hops_df.shape

(3268,)

In [6]:
misc_df = pd.read_csv("MISC.csv")
print (misc_df.head())

   Recipe_ID                          NAME  VERSION    TYPE        USE  TIME  \
0       1633          pure vanilla extract        1  Flavor       Boil   0.0   
1       1633          pure vanilla extract        1  Flavor   Bottling   0.0   
2       1633                yeast nutrient        1   Other       Boil  15.0   
3       1633                     whirlfloc        1  Fining       Boil  15.0   
4       1633  Vanilla beans - in 2oz Vodka        1   Other  Secondary   0.0   

     AMOUNT  AMOUNT_IS_WEIGHT  
0  0.059147             False  
1  0.029574             False  
2  0.004929             False  
3  1.000000              True  
4  1.000000              True  


In [7]:
unique_misc_df = misc_df['NAME'].unique()
unique_misc_df.shape

(10255,)

In [8]:
yeast_df = pd.read_csv('YEAST.csv')
print(yeast_df.head())

   Recipe_ID                                NAME  VERSION TYPE    FORM  \
0       1633                         Kölsch 2565        1  Ale  Liquid   
1      16367                   American Ale 1056        1  Ale  Liquid   
2       5920     Safale - English Ale Yeast S-04        1  Ale     Dry   
3       5916     Safale - English Ale Yeast S-04        1  Ale     Dry   
4      89534  Safbrew - Specialty Ale Yeast T-58        1  Ale     Dry   

   AMOUNT PRODUCT_ID          LABORATORY  ATTENUATION FLOCCULATION  \
0    0.10       2565              Wyeast         76.0          Low   
1    0.10       1056              Wyeast         75.0       Medium   
2    0.11       S-04  Fermentis / Safale         75.0         High   
3    0.11       S-04  Fermentis / Safale         75.0         High   
4    0.11       T-58  Fermentis / Safale         80.0         High   

   MIN_TEMPERATURE  MAX_TEMPERATURE AMOUNT_IS_WEIGHT  
0        13.333333        21.111111              NaN  
1        15.555556      

In [9]:
unique_yeast_df = yeast_df['NAME'].unique()
unique_yeast_df.shape

(1727,)

In [11]:
recipe_df = pd.read_csv('recipeData.csv', engine = 'python', encoding = 'ISO-8859-1')
recipe_df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,BoilGravity,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,1.038,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,1.070,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,1.050,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0


In [13]:
id_df = pd.read_csv('recipe_id.csv')
id_df.head()

,Recipe ID
0,1633
1,16367
2,5920
3,5916
4,89534


In [14]:
recipe_ID_df = recipe_df.join(id_df['Recipe ID'])
recipe_ID_df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,Efficiency,MashThickness,SugarScale,BrewMethod,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId,Recipe ID
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,70.0,NaN,Specific Gravity,All Grain,NaN,17.78,corn sugar,4.5 oz,116.0,1633
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,955.0,16367
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,70.0,NaN,Specific Gravity,extract,NaN,NaN,NaN,NaN,NaN,5920
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,70.0,NaN,Specific Gravity,All Grain,NaN,NaN,NaN,NaN,NaN,5916
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,72.0,NaN,Specific Gravity,All Grain,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0,89534


In [15]:
review_df = pd.read_csv('reviewData_all.csv', engine = 'python', encoding = 'ISO-8859-1')
review_df.head()

,url,rating,reviews,calories,carbs
0,https://www.brewersfriend.com/homebrew/recipe/...,4.78,18.0,181 calories,18.2 g
1,https://www.brewersfriend.com/homebrew/recipe/...,4.69,16.0,277 calories,28.6 g
2,https://www.brewersfriend.com/homebrew/recipe/...,4.67,9.0,209 calories,21.8 g
3,https://www.brewersfriend.com/homebrew/recipe/...,4.70,10.0,202 calories,21.5 g
4,https://www.brewersfriend.com/homebrew/recipe/...,3.80,5.0,189 calories,17.2 g


In [16]:
review_df['calories']= review_df['calories'].fillna("-1").apply(lambda x: int(x.split()[0]))

In [17]:
review_df['carbs']= review_df['carbs'].fillna("-1").apply(lambda x: float(x.split()[0]))

In [18]:
review_no_url_df = review_df.drop('url', axis=1)

In [19]:
review_no_url_df.head()

,rating,reviews,calories,carbs
0,4.78,18.0,181,18.2
1,4.69,16.0,277,28.6
2,4.67,9.0,209,21.8
3,4.70,10.0,202,21.5
4,3.80,5.0,189,17.2


In [20]:
recipe_ID_review_df = pd.concat([recipe_ID_df, review_no_url_df], axis = 1)
recipe_ID_review_df

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId,Recipe ID,rating,reviews,calories,carbs
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,NaN,17.78,corn sugar,4.5 oz,116.0,1633,4.78,18.0,181,18.2
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,NaN,NaN,NaN,NaN,955.0,16367,4.69,16.0,277,28.6
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,NaN,NaN,NaN,NaN,5920,4.67,9.0,209,21.8
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,NaN,NaN,NaN,NaN,5916,4.70,10.0,202,21.5
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0,89534,3.80,5.0,189,17.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73856,73857,Amber Alfie 2,/homebrew/recipe/view/609673/amber-alfie-2,British Strong Ale,36,21.00,1.052,1.012,5.22,39.73,...,1.25,20.00,NaN,NaN,59658.0,609673,NaN,NaN,159,15.8
73857,73858,Rye IPA,/homebrew/recipe/view/610955/rye-ipa,Specialty IPA: Rye IPA,151,10.00,1.059,1.010,6.37,59.21,...,0.50,22.00,NaN,NaN,NaN,610955,NaN,NaN,179,15.4
73858,73859,SK-Kölsch,/homebrew/recipe/view/586891/,NaN,111,16.00,1.046,1.008,5.01,25.42,...,0.35,17.00,sucrose,140 g,82450.0,586891,NaN,NaN,139,12.1
73859,73860,Flata Rødkløver,/homebrew/recipe/view/603788/,Irish Red Ale,92,24.00,1.051,1.014,4.82,25.65,...,NaN,18.00,Sukkerlake,5 g sukker/l,NaN,603788,NaN,NaN,132,14.5


In [21]:
# Save the combined data as a csv file for later: ___Data_all.csv

recipe_ID_review_df.to_csv('recipe_id_review_Data_all.csv')

In [23]:
recipe_ID_review_df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId,Recipe ID,rating,reviews,calories,carbs
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,NaN,17.78,corn sugar,4.5 oz,116.0,1633,4.78,18.0,181,18.2
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,NaN,NaN,NaN,NaN,955.0,16367,4.69,16.0,277,28.6
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,NaN,NaN,NaN,NaN,5920,4.67,9.0,209,21.8
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,NaN,NaN,NaN,NaN,5916,4.70,10.0,202,21.5
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0,89534,3.80,5.0,189,17.2


In [24]:
refined_recipe_df = recipe_ID_review_df[recipe_ID_review_df['rating'].notna()]

In [25]:
refined_recipe_df.head()

,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,IBU,...,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId,Recipe ID,rating,reviews,calories,carbs
0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,...,NaN,17.78,corn sugar,4.5 oz,116.0,1633,4.78,18.0,181,18.2
1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,...,NaN,NaN,NaN,NaN,955.0,16367,4.69,16.0,277,28.6
2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,59.25,...,NaN,NaN,NaN,NaN,NaN,5920,4.67,9.0,209,21.8
3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,54.48,...,NaN,NaN,NaN,NaN,NaN,5916,4.70,10.0,202,21.5
4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,...,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0,89534,3.80,5.0,189,17.2


In [26]:
refined_recipe_df.shape

(2192, 28)

In [27]:
# Save the refined data as a csv file for later: ___Data_refined.csv

refined_recipe_df.to_csv('recipe_id_review_Data_refined.csv')

In [10]:
refined_recipe_df= pd.read_csv('recipe_id_review_Data_refined.csv')
refined_recipe_df.head()

,Unnamed: 0,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,...,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId,Recipe ID,rating,reviews,calories,carbs
0,0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,...,NaN,17.78,corn sugar,4.5 oz,116.0,1633,4.78,18.0,181,18.2
1,1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,...,NaN,NaN,NaN,NaN,955.0,16367,4.69,16.0,277,28.6
2,2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,...,NaN,NaN,NaN,NaN,NaN,5920,4.67,9.0,209,21.8
3,3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,...,NaN,NaN,NaN,NaN,NaN,5916,4.70,10.0,202,21.5
4,4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,...,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0,89534,3.80,5.0,189,17.2


In [11]:
refined_recipe_df.shape

(2192, 29)

In [12]:
'''Recipe ID 21783 is a root beer recipe and is outside the scope of this project. Recipe ID 314560 is not a real recipe.'''
refined_recipe_df = refined_recipe_df[refined_recipe_df['Recipe ID'] != 21783]
refined_recipe_df = refined_recipe_df[refined_recipe_df['Recipe ID'] != 314560]


In [13]:
refined_recipe_df.shape

(2190, 29)

In [14]:
refined_recipe_df

,Unnamed: 0,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,...,PitchRate,PrimaryTemp,PrimingMethod,PrimingAmount,UserId,Recipe ID,rating,reviews,calories,carbs
0,0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,...,NaN,17.78,corn sugar,4.5 oz,116.0,1633,4.78,18.0,181,18.2
1,1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,...,NaN,NaN,NaN,NaN,955.0,16367,4.69,16.0,277,28.6
2,2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,...,NaN,NaN,NaN,NaN,NaN,5920,4.67,9.0,209,21.8
3,3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,...,NaN,NaN,NaN,NaN,NaN,5916,4.70,10.0,202,21.5
4,4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,...,NaN,19.00,Sukkerlake,6-7 g sukker/l,18325.0,89534,3.80,5.0,189,17.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,73375,73376,Käftsmälls IPA,/homebrew/recipe/view/606909/,American IPA,7,10.00,1.064,1.012,6.85,...,NaN,20.00,NaN,NaN,NaN,606909,5.00,1.0,217,19.2
2188,73432,73433,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/618500/zombie-dust-clone...,American IPA,7,22.70,1.061,1.014,6.13,...,1.25,20.00,NaN,NaN,NaN,618500,5.00,1.0,184,18.4
2189,73549,73550,German pils,/homebrew/recipe/view/616365/german-pils,German Pils,80,27.00,1.050,1.011,5.06,...,NaN,NaN,NaN,NaN,NaN,616365,5.00,1.0,153,14.8
2190,73628,73629,Magelan IPA 2018.,/homebrew/recipe/view/134193/magelan-ipa-2018-,English IPA,65,30.00,1.060,1.011,6.39,...,0.75,23.00,NaN,NaN,NaN,134193,5.00,1.0,180,16.1


In [29]:
len(refined_recipe_df['Name'].unique())

2113

In order to integrade the ingredient data, classes were developed. 

Below are the steps taken to gather that information

First, unique ingredient tables were extracted from each of the ingredient tables. The first example is taking each of the unique fermentables out of the fermentables table. The same process was done for the hops, misc, and yeast data.

Classes were developed to help integrate the ingredient data.

In [15]:
class Fermentable:
    
#    id = 0
    recipe_array = []
    percent_bill = 0
    main_ingredient = False
    origin = ""
    
    def __init__(self, name, amount, ferm_type):
        self.name = name
        self.amount = amount
        self.ferm_type = ferm_type
        
    def __str__(self):
        if self.main_ingredient == False:
            print(str(self.name) + ' is not the main ingredient. Percentage is ' + str(self.percent_bill) + '%.')
        else:
            print(str(self.name) + ' is the main ingredient. Percentage is ' + str(self.percent_bill) + '%.')
    
    def set_name(self, new_name):
        self.name = new_name
    
    def set_amount(self, new_amount):
        self.amount = new_amount
    
    def set_id(self, new_id):
        self.id = new_id
        
    def set_recipe_array(self, new_recipe_array):
        self.recipe_array = new_recipe_array
    
    def set_percent_bill(self, new_percent_bill):
        self.percent_bill = new_percent_bill
    
    def set_main_ingredient(self, new_main_ingredient):
        self.main_ingredient = new_main_ingredient

    def set_ferm_type(self, new_ferm_type):
        self.ferm_type = new_ferm_type
    
    def set_origin(self, new_origin):
        self.origin = new_origin
        
        
    def get_name(self):
        return self.name
    
    def get_amount(self):
        return self.amount
    
    def get_id(self):
        return self.id
        
    def get_recipe_array(self):
        return self.recipe_array
    
    def get_percent_bill(self):
        return self.percent_bill
    
    def get_main_ingredient(self):
        return self.main_ingredient
    
    def get_ferm_type(self):
        return self.ferm_type
    
    def get_origin(self):
        return self.origin

In [16]:
class Hops:
    
#   id = 0
    recipe_array = []
   
    def __init__(self, name, amount, alpha, use, time):
        self.name = name
        self.amount = amount
        self.alpha = alpha
        self.hop_use = use
        self.hop_time = time
    
    def __str__(self):
        return 'This hops is ' + str(self.name) + ' and has ' + str(self.amount) + ' amount and ' + str(self.alpha) + ' alpha.'
    
    def set_name(self, new_name):
        self.name = new_name
    
    def set_amount(self, new_amount):
        self.amount = new_amount
    
    def set_id(self, new_id):
        self.id = new_id
        
    def set_recipe_array(self, new_recipe_array):
        self.recipe_array = new_recipe_array
    
    def set_alpha(self, new_alpha):
        self.alpha = new_alpha

    def set_hop_use(self, new_hop_use):
        self.hop_use = new_hop_use        
        
    def set_hop_time(self, new_time):
        self.hop_time = new_time
        
        
    def get_name(self):
        return self.name
    
    def get_amount(self):
        return self.amount
    
    def get_id(self):
        return self.id
        
    def get_recipe_array(self):
        return self.recipe_array
    
    def get_alpha(self):
        return self.alpha
    
    def get_hop_use(self):
        return self.hop_use 
    
    def get_hop_time(self):
        return self.hop_time

In [17]:
class Yeast:
    
#    id = 0
    recipe_array = []
   
    def __init__(self, name, laboratory, flocculation, form, yeast_type):
        self.name = name
        self.laboratory = laboratory
        self.flocculation = flocculation
        self.form = form
        self.yeast_type = yeast_type
    
    def set_name(self, new_name):
        self.name = new_name
    
    def set_laboratory(self, new_laboratory):
        self.laboratory = new_laboratory
    
    def set_id(self, new_id):
        self.id = new_id
        
    def set_recipe_array(self, new_recipe_array):
        self.recipe_array = new_recipe_array
    
    def set_flocculation(self, new_flocculation):
        self.flocculation = new_flocculation
   
    def set_form(self, new_form):
        self.form = new_form        

    def set_yeast_type(self, new_yeast_type):
        self.yeast_type = new_yeast_type        
        
        
    def get_name(self):
        return self.name
    
    def get_laboratory(self):
        return self.laboratory
    
    def get_id(self):
        return self.id
        
    def get_recipe_array(self):
        return self.recipe_array
    
    def get_flocculation(self):
        return self.flocculation
   
    def get_form(self):
        return self.form
    
    def get_yeast_type(self):
        return self.yeast_type

In [18]:
class Misc:
   
    def __init__(self, name, amount, id, recipe_array):
        self.name = name
        self.amount = amount
        self.id = id
        self.recipe_array = recipe_array
    
    
    def set_name(self, new_name):
        self.name = new_name
    
    def set_amount(self, new_amount):
        self.amount = new_amount
    
    def set_id(self, new_id):
        self.id = new_id
        
    def set_recipe_array(self, new_recipe_array):
        self.recipe_array = new_recipe_array
    
        
    def get_name(self):
        return self.name
    
    def get_amount(self):
        return self.amount
    
    def get_id(self):
        return self.id
        
    def get_recipe_array(self):
        return self.recipe_array

In [19]:
class Recipe:
    
    fermentable = []
    hops = []
    yeast = []
   
    def __init__(self, name, id, rating):
        self.name = name
        self.id = id
        self.rating = rating
        
    def __str__(self):
        return 'This recipe is ' + str(self.name) + ' and has the ID ' + str(self.id) + ' and rating ' + str(self.rating) + '. It has ' + str(len(self.fermentable)) + ' fermentables, ' + str(len(self.hops)) + ' hops, and the yeast is ' + str(self.yeast.name) + '.'
    
    def set_name(self, new_name):
        self.name = new_name
    
    def set_amount(self, new_amount):
        self.amount = new_amount
    
    def set_id(self, new_id):
        self.id = new_id
        
    def set_rating(self, new_rating):
        self.rating = new_rating
    
    def set_fermentable(self, new_fermentable):
        self.fermentable = new_fermentable

    def set_hops(self, new_hops):
        self.hops = new_hops
 
    def set_yeast(self, new_yeast):
        self.yeast = new_yeast

    def set_extract_percent(self, new_percent):
        self.extract_percent = new_percent

    def set_grain_percent(self, new_percent):
        self.grain_percent = new_percent

    def set_sugar_percent(self, new_percent):
        self.sugar_percent = new_percent

    def set_dry_extract_percent(self, new_percent):
        self.dry_extract_percent = new_percent        

    def set_adjunct_percent(self, new_percent):
        self.adjunct_percent = new_percent

    def set_total_ingredients(self, new_percent):
        self.total_ingredients = new_percent
        
    def set_extract_total(self, total):
        self.extract_total = total
        
    def set_grain_total(self, total):
        self.grain_total = total

    def set_sugar_total(self, total):
        self.sugar_total = total

    def set_dry_extract_total(self, total):
        self.dry_extract_total = total

    def set_adjunct_total(self, total):
        self.adjunct_total = total

    def set_hop_aroma_total(self, total):
        self.hop_aroma_total = total
        
    def set_hop_boil_total(self, total):
        self.hop_boil_total = total
        
    def set_hop_dry_total(self, total):
        self.hop_dry_total = total
        
    def set_hop_first_wort_total(self, total):
        self.hop_first_wort_total = total
        
    def set_hop_mash_total(self, total):
        self.hop_mash_total = total
        
        
    def get_name(self):
        return self.name
    
    def get_amount(self):
        return self.amount
    
    def get_id(self):
        return self.id
        
    def get_rating(self):
        return self.rating
    
    def get_fermentable(self):
        return self.fermentable

    def get_hops(self):
        return self.hops
 
    def get_yeast(self):
        return self.yeast
   
    def get_extract_percent(self):
        return self.extract_percent

    def get_grain_percent(self):
        return self.grain_percent

    def get_sugar_percent(self):
        return self.sugar_percent

    def get_dry_extract_percent(self):
        return self.dry_extract_percent        

    def get_adjunct_percent(self):
        return self.adjunct_percent

    def get_total_ingredients(self):
        return self.total_ingredients
    
    def get_extract_total(self):
        return self.extract_total
        
    def get_grain_total(self):
        return self.grain_total

    def get_sugar_total(self):
        return self.sugar_total

    def get_dry_extract_total(self):
        return self.dry_extract_total

    def get_adjunct_total(self):
        return self.adjunct_total
    
    def get_hop_aroma_total(self):
        return self.hop_aroma_total
        
    def get_hop_boil_total(self):
        return self.hop_boil_total
        
    def get_hop_dry_total(self):
        return self.hop_dry_total
        
    def get_hop_first_wort_total(self):
        return self.hop_first_wort_total
        
    def get_hop_mash_total(self):
        return self.hop_mash_total

In [20]:
class IngredientController:
    
    """Using a Recipe ID to find the yeast used in specified recipe."""
    def find_yeast_by_recipe_id(recipe_id):
        temp = yeast_df[yeast_df['Recipe_ID'] == recipe_id]
        # print(temp)
        yeast = Yeast(temp.iloc[0]['NAME'], temp.iloc[0]['LABORATORY'], temp.iloc[0]['FLOCCULATION'], temp.iloc[0]['FORM'], temp.iloc[0]['TYPE'])
        # print(yeast)
        return yeast
    
    """Using a Recipe ID to find the hops used in specified recipe"""
    def find_hops_by_recipe_id(recipe_id):
        temp = hops_df[hops_df['Recipe_ID'] == recipe_id]
        index = temp.index
        number_of_rows = len(index)
        hops_array = []
        for i in range(number_of_rows):
            row = temp.iloc[i]
            hops = Hops(row['NAME'], row['AMOUNT'], row['ALPHA'], row['USE'], row['TIME'])
            hops_array.append(hops)
        return hops_array
    
    """Using a Recipe ID to find the fermentable used in specified recipe"""
    def find_ferm_by_recipe_id(recipe_id):
        temp = ferm_df[ferm_df['Recipe_ID'] == recipe_id]
        index = temp.index
        number_of_rows = len(index)
        ferm_array = []
        for i in range(number_of_rows):
            row = temp.iloc[i]
            ferm = Fermentable(row['NAME'], row['AMOUNT'], row['TYPE'])
            ferm_array.append(ferm)
        #IngredientController.find_percent_bill(ferm_array)
        #IngredientController.find_main_ingredient(ferm_array)
        return ferm_array

In [21]:
class RecipeController:
 
    """Using the list of fermentables find the percentage of each fermentable used in specified recipe from 0 to 1"""
    def find_percent_bill(ferm_array):
        total = 0
        new_percent = 0
        first_time = True
        main_ingredient_index = 0
        for i in range(len(ferm_array)):
            total = total + ferm_array[i].amount
        for i in range(len(ferm_array)):
            new_percent = ferm_array[i].amount / total
            ferm_array[i].set_percent_bill(new_percent)
            if first_time == True:
                first_time = False
            else: 
                if ferm_array[main_ingredient_index].percent_bill < ferm_array[i].percent_bill:
                    main_ingredient_index = i
        ferm_array[main_ingredient_index].set_main_ingredient(True)
        return ferm_array
    
    def find_main_ingredient(recipe_id):
        recipe = RecipeController.find_recipe_by_id(recipe_id)
        main_ingredient = ""
        for i in range(len(recipe.fermentable)):
            if recipe.fermentable[i].main_ingredient == True:
                main_ingredient = recipe.fermentable[i].name
        return main_ingredient
                
    
    """determine if recipe has a rating"""
    
    """determine rating of unrated recipe"""
    
    """Retrieve ingredients for recipe"""
    
    """find recipe by recipe_ID"""
    def find_recipe_by_id(recipe_id):
        temp = refined_recipe_df[refined_recipe_df['Recipe ID'] == recipe_id]
        recipe = Recipe(temp.iloc[0]['Name'], temp.iloc[0]['Recipe ID'], temp.iloc[0]['rating'])
        ferm_array = IngredientController.find_ferm_by_recipe_id(recipe_id)
        recipe.set_fermentable(ferm_array)
        RecipeController.find_percent_bill(ferm_array)
        hops_array = IngredientController.find_hops_by_recipe_id(recipe_id)
        recipe.set_hops(hops_array)
        yeast_array = IngredientController.find_yeast_by_recipe_id(recipe_id)
        recipe.set_yeast(yeast_array)
        #print(recipe)
        return recipe

    def make_table(recipe_id):
        percents = RecipeController.find_percent_ferm_type_by_id(recipe_id)
        adjunct = percents[0]
        dry = percents[1]
        extract = percents[2]
        grain = percents[3]
        sugar = percents[4]
        main_ingredient = RecipeController.find_main_ingredient(recipe_id)
        data = [recipe_id, adjunct, dry, extract, grain, sugar, main_ingredient]
        partial_table = pd.DataFrame(data, columns = ['Recipe ID', 'Adjunct Percent', 'Dry Extract Percent', 'Extract Percent', 'Grain Percent', 'Sugar Percent', 'Main Ingredient' ])
        return partial_table
        
    def find_percent_ferm_type_by_id(recipe_id):
        
        """Determine the percentages of each type of fermentables using the recipe ID and ferm_array"""

        recipe = RecipeController.find_recipe_by_id(recipe_id)
                     
        extract_tot = 0
        grain_tot = 0
        sugar_tot = 0
        dry_tot = 0
        adjunct_tot = 0
        total_ingredients = 0
        data = []
        
        for i in range(len(recipe.fermentable)):
            if recipe.fermentable[i].ferm_type == 'Dry Extract':
                dry_tot = dry_tot + recipe.fermentable[i].amount
                #print("dry extract: ", dry_tot)
            elif recipe.fermentable[i].ferm_type == 'Sugar':
                sugar_tot = sugar_tot + recipe.fermentable[i].amount
                #print("sugar: ", sugar_tot)
            elif recipe.fermentable[i].ferm_type == 'Adjunct':
                adjunct_tot = adjunct_tot + recipe.fermentable[i].amount
                #print("adjunct: ", adjunct_tot)
            elif recipe.fermentable[i].ferm_type == 'Grain':
                grain_tot = grain_tot + recipe.fermentable[i].amount
                #print("grain: ", grain_tot)
            else: #recipe.fermentable[i].ferm_type == 'Extract'
                extract_tot = extract_tot + recipe.fermentable[i].amount
                #print("extract: ", extract_tot)
        
        total_ingredients = dry_tot + sugar_tot + adjunct_tot + grain_tot + extract_tot
        
        extract_percent = extract_tot / total_ingredients
        grain_percent = grain_tot / total_ingredients
        sugar_percent = sugar_tot / total_ingredients
        dry_percent = dry_tot / total_ingredients
        adjunct_percent = adjunct_tot / total_ingredients
        
        recipe.set_extract_total(extract_percent)
        recipe.set_grain_total(grain_percent)
        recipe.set_sugar_total(sugar_percent)
        recipe.set_dry_extract_total(dry_percent)
        recipe.set_adjunct_total(adjunct_percent)
        
        info = [recipe.adjunct_total, recipe.dry_extract_total, recipe.extract_total, recipe.grain_total, recipe.sugar_total]
        
        return info
    
    def find_percent_hop_use_by_id(recipe_id):
        
        """Determine the percentages of each use-type for each hops using the recipe ID and hops_array"""

        recipe = RecipeController.find_recipe_by_id(recipe_id)
                     
        aroma_tot = 0
        boil_tot = 0
        first_tot = 0
        dry_tot = 0
        mash_tot = 0
        total_hops = 0
        
        for i in range(len(recipe.hops)):
            if recipe.hops[i].hop_use == 'Aroma':
                aroma_tot = aroma_tot + recipe.hops[i].amount
                #print("dry extract: ", dry_tot)
            elif recipe.hops[i].hop_use == 'Boil':
                boil_tot = boil_tot + recipe.hops[i].amount
                #print("sugar: ", sugar_tot)
            elif recipe.hops[i].hop_use == 'Dry Hop':
                dry_tot = dry_tot + recipe.hops[i].amount
                #print("adjunct: ", adjunct_tot)
            elif recipe.hops[i].hop_use == 'First Wort':
                first_tot = first_tot + recipe.hops[i].amount
                #print("grain: ", grain_tot)
            else: #recipe.hops[i].hop_use == 'Mash'
                mash_tot = mash_tot + recipe.hops[i].amount
                #print("extract: ", extract_tot)
        
        total_hops = dry_tot + aroma_tot + boil_tot + first_tot + mash_tot
        
        if total_hops != 0:
            aroma_percent = aroma_tot / total_hops
            first_percent = first_tot / total_hops
            boil_percent = boil_tot / total_hops
            dry_percent = dry_tot / total_hops
            mash_percent = mash_tot / total_hops

            recipe.set_hop_aroma_total(aroma_percent)
            recipe.set_hop_boil_total(boil_percent)
            recipe.set_hop_dry_total(dry_percent)
            recipe.set_hop_first_wort_total(first_percent)
            recipe.set_hop_mash_total(mash_percent)

            info = [recipe.hop_aroma_total, recipe.hop_boil_total, recipe.hop_dry_total, recipe.hop_first_wort_total, recipe.hop_mash_total]

            return info
        else:
            info = [0, 0, 0, 0, 0]

            return info

    def gather_hop_amounts(recipe_ID):
        recipe = RecipeController.find_recipe_by_id(recipe_ID)
        info = [['Hop_Name', 'Hop_Amount', 'Hop_Use']]
        for i in recipe.hops:
            temp_hop = [i.name, i.amount, i.hop_use]
            info.append(temp_hop)
        return info

In [76]:
#create blank ferm ingredients table

# recipe_id column and the percent Ferm_type% columns first

f = {'Recipe_ID': 0, 'Percent_Adjunct(%)': 0,'Percent_Dry_Extract(%)': 0, 'Percent_Extract(%)': 0,'Percent_Grain(%)': 0, 'Percent_Sugar(%)': 0,'Percent_Ferm_Total(%)': 0, 'Main_Ingredient':''}
f_ing_df = pd.DataFrame(f, index= [0])
f_ing_df.head()

,Recipe_ID,Percent_Adjunct(%),Percent_Dry_Extract(%),Percent_Extract(%),Percent_Grain(%),Percent_Sugar(%),Percent_Ferm_Total(%),Main_Ingredient
0,0,0,0,0,0,0,0,


In [39]:
def add_fermentable_ing(df,  name):
    col1 = name
    
    amt = 0
    empty_bool = False
    empty_type = ''
    
    df[col1] = amt
   
    return df

def add_hops_ing(df, name):
    col1 = name
    col2 = 'alpha_' + str(name)
    col3 = 'use_Aroma_' + str(name)
    col4 = 'use_Boil_' + str(name)
    col5 = 'use_DryHop_' + str(name)
    col6 = 'use_FirstWort_' + str(name)
    col7 = 'use_Mash_' + str(name)
    
    amt = 0
    empty_bool = False
    empty_type = ''
    
    df[col1] = empty_bool
    df[col2] = amt
    df[col3] = amt
    df[col4] = amt
    df[col5] = amt
    df[col6] = amt
    df[col7] = amt
    
    return df

def add_yeast_name(df, name):
    
    col1 = name
    empty_bool = False
    df[col1]= empty_bool
    return df

def add_yeast_lab(df, lab):
    col1 = 'lab_'+ str(lab)
    empty_bool = False
    df[col1] = empty_bool
    return df

def add_yeast_type(df, ytype):
    col1 = 'ytype_' + str(ytype)
    empty_bool = False
    df[col1] = empty_bool
    return df

def add_percents_to_table(recipe_id):
        percents = RecipeController.find_percent_ferm_type_by_id(recipe_id)
        adjunct = percents[0]
        dry = percents[1]
        extract = percents[2]
        grain = percents[3]
        sugar = percents[4]
        main_ingredient = RecipeController.find_main_ingredient(recipe_id)
        data = [recipe_id, adjunct, dry, extract, grain, sugar, main_ingredient]
        return data
    
def add_main_ing(df, name):
    
    col1 = 'main_ing_' + str(name)
    empty_bool = False
    df[col1] = empty_bool
    
    return df

In [78]:
for i in ferm_df['NAME'].unique():
    
    f_ing_unique_df = add_fermentable_ing(f_ing_df, i)


<ipython-input-77-804ffff9ca06>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col1] = amt


In [79]:
f_ing_unique_df.head()

,Recipe_ID,Percent_Adjunct(%),Percent_Dry_Extract(%),Percent_Extract(%),Percent_Grain(%),Percent_Sugar(%),Percent_Ferm_Total(%),Main_Ingredient,Pale 2-Row,White Wheat,...,Raw Barley,Gas Hog Rice Malt,Amber Rice Malt,Ireland - Ale Malt,Home Smoked Vienna Malt. (cherry wood),American - Blackprinz Malt,SUPERIOR PALE ALE MALT,Dry Malt Extract - Pale Ale,Best Malz Rye,JWM Traditional Ale Malt
0,0,0,0,0,0,0,0,,0,0,...,0,0,0,0,0,0,0,0,0,0


In [130]:
def populate_table(in_df, recipe_ID):
    column_dict = {'Recipe_ID': 0, 'Percent_Adjunct(%)': 0,'Percent_Dry_Extract(%)': 0, 'Percent_Extract(%)': 0,'Percent_Grain(%)': 0, 'Percent_Sugar(%)': 0,'Percent_Ferm_Total(%)': 0, 'Main_Ingredient':''}
    out_df = pd.DataFrame(column_dict, index= [recipe_ID])
    
    temp_list = add_percents_to_table(recipe_ID)
    
    out_df['Recipe_ID'] = temp_list[0]
    out_df['Percent_Adjunct(%)'] = temp_list[1]
    out_df['Percent_Dry_Extract(%)'] = temp_list[2]
    out_df['Percent_Extract(%)'] = temp_list[3]
    out_df['Percent_Grain(%)'] = temp_list[4]
    out_df['Percent_Sugar(%)'] = temp_list[5]
    out_df['Percent_Ferm_Total(%)'] = sum(temp_list[1:6])
    out_df['Main_Ingredient'] = temp_list[6]
    
    temp_recipe = RecipeController.find_recipe_by_id(recipe_ID)
    
    ferm_array = []
    for i in temp_recipe.fermentable:
        ferm_array.append(i)
        
    for j in ferm_array:
        out_df[j.name] = j.amount
    
    frames = [in_df, out_df]
    result_df = pd.concat(frames, axis=0)
    
    return result_df


def populate_main_ing_table(in_df, recipe_ID):
    main_ingredient = in_df.loc[recipe_ID, "Main_Ingredient"]
    col = in_df.filter(like = main_ingredient)
    print(col)
    in_df.loc[recipe_ID, col] = True
    
    return in_df

In [81]:
a = 0
for i in refined_recipe_df['Recipe ID']:
    f_ing_df = populate_table(f_ing_df, i)

    a += 1
    if a % 100 == 0:
        print("Just made row #" +str(a))
        
f_ing_df.head()

Just made row #100
Just made row #200
Just made row #300
Just made row #400
Just made row #500
Just made row #600
Just made row #700
Just made row #800
Just made row #900
Just made row #1000
Just made row #1100
Just made row #1200
Just made row #1300
Just made row #1400
Just made row #1500
Just made row #1600
Just made row #1700
Just made row #1800
Just made row #1900
Just made row #2000
Just made row #2100


,Recipe_ID,Percent_Adjunct(%),Percent_Dry_Extract(%),Percent_Extract(%),Percent_Grain(%),Percent_Sugar(%),Percent_Ferm_Total(%),Main_Ingredient,Pale 2-Row,White Wheat,...,Raw Barley,Gas Hog Rice Malt,Amber Rice Malt,Ireland - Ale Malt,Home Smoked Vienna Malt. (cherry wood),American - Blackprinz Malt,SUPERIOR PALE ALE MALT,Dry Malt Extract - Pale Ale,Best Malz Rye,JWM Traditional Ale Malt
0,0,0.00000,0.000000,0.0,0.000000,0.00000,0.0,,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1633,1633,0.06383,0.000000,0.0,0.872340,0.06383,1.0,Pale 2-Row,2.381360,0.907185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16367,16367,0.00000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,6.803886,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5920,5920,0.00000,0.705882,0.0,0.294118,0.00000,1.0,Dry Malt Extract - Extra Light,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5916,5916,0.00000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,5.329710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
f_ing_df = f_ing_df.iloc[1:, :]

Recipe ID #328022 did not contain yeast data within the recipe. I researched and found a similar clone recipe (#2918022) within the database that is unrated but nearly identical that uses the yeast Saflager S-23. This was checked against an external website, "speidels-braumeister.de/en/breweing-recipes/pilsner.html", that brews a similar Pilsner style beer.

Recipe ID #450524 did not contain a yeast within the recipe. Researched in a similar fashion and found a few resources for the White Zombie Pale Ale clone using White Labs#WLP400 yeast.

Recipe ID #545272 did not contain yeast data within the yeast section but did have it within the comment section. This yeast was Ebbegarden Kveik - Valkyrie from WHC labs.

Recipe ID #604836 did not contain a yeast within the recipe. Researched similar Black IPA recipes and discovered Safale US-05 is often used.

In [83]:
f_ing_df

,Recipe_ID,Percent_Adjunct(%),Percent_Dry_Extract(%),Percent_Extract(%),Percent_Grain(%),Percent_Sugar(%),Percent_Ferm_Total(%),Main_Ingredient,Pale 2-Row,White Wheat,...,Raw Barley,Gas Hog Rice Malt,Amber Rice Malt,Ireland - Ale Malt,Home Smoked Vienna Malt. (cherry wood),American - Blackprinz Malt,SUPERIOR PALE ALE MALT,Dry Malt Extract - Pale Ale,Best Malz Rye,JWM Traditional Ale Malt
1633,1633,0.063830,0.000000,0.0,0.872340,0.06383,1.0,Pale 2-Row,2.381360,0.907185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16367,16367,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,6.803886,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5920,5920,0.000000,0.705882,0.0,0.294118,0.00000,1.0,Dry Malt Extract - Extra Light,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5916,5916,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,5.329710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89534,89534,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Castle Malting Pilsen 2RP/2RS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606909,606909,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale Ale,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618500,618500,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Maris Otter Pale,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616365,616365,0.050000,0.000000,0.0,0.950000,0.00000,1.0,Pilsner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134193,134193,0.021277,0.000000,0.0,0.978723,0.00000,1.0,Pilsner,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
f_ing_df.isnull().all(axis=0)

Recipe_ID                      False
Percent_Adjunct(%)             False
Percent_Dry_Extract(%)         False
Percent_Extract(%)             False
Percent_Grain(%)               False
                               ...  
American - Blackprinz Malt      True
SUPERIOR PALE ALE MALT          True
Dry Malt Extract - Pale Ale     True
Best Malz Rye                   True
JWM Traditional Ale Malt        True
Length: 3978, dtype: bool

In [85]:
f_ing_df = f_ing_df.dropna(axis=1, how='all')
f_ing_df.shape

(2192, 446)

In [86]:
f_ing_df.to_csv('ing_table_ferm.csv')

In [137]:
ing_table_ferm_df = pd.read_csv('ing_table_ferm.csv')
ing_table_ferm_df = ing_table_ferm_df.iloc[:, 1:]

In [138]:
main_ing = ing_table_ferm_df['Main_Ingredient']
unique_main_ing = main_ing.unique()

In [139]:
unique_main_ing

array(['Pale 2-Row', 'Dry Malt Extract - Extra Light',
       'Castle Malting Pilsen 2RP/2RS',
       'Thomas Fawcett Pale Ale Malt (Maris Otter)',
       'Liquid Malt Extract - Light', 'Pilsner', 'Pale Ale',
       'Maris Otter Pale', 'Thomas Fawcett Maris Otter Pale Malt',
       'Dry Malt Extract - Light', 'Best Pilsener',
       'Briess Pilsen Light Liquid Malt Extract', 'Honey',
       'Dingemans - Pilsen', 'Dry Malt Extract - Pilsen', 'Best Pale Ale',
       'Warminster Floor Malted Maris Otter Pale', 'Amber', 'White Wheat',
       'Pilsen', 'Wheat Malt', 'Golden Promise', 'Pearl', '2-Row Malt',
       'Northern Brewer Gold LME', 'Lactose (Milk Sugar)',
       'Liquid Malt Extract - Amber', 'Base Malt',
       'Liquid Malt Extract - Munich', 'Munich - Light 10L',
       'Liquid Malt Extract - Extra Light', 'Wheat', 'Best Munich',
       'Pilsner Malt', 'Cara Malt', 'Smoked Malt', 'Lager',
       'Bohemian Pilsner', 'Best ale', 'British Pale Malt (Marris Otter)',
       'Liquid Ma

In [140]:
main_ing_df = ing_table_ferm_df.loc[:, ['Recipe_ID', 'Main_Ingredient']]
main_ing_df

,Recipe_ID,Main_Ingredient
0,1633,Pale 2-Row
1,16367,Pale 2-Row
2,5920,Dry Malt Extract - Extra Light
3,5916,Pale 2-Row
4,89534,Castle Malting Pilsen 2RP/2RS
...,...,...
2187,606909,Pale Ale
2188,618500,Maris Otter Pale
2189,616365,Pilsner
2190,134193,Pilsner


In [141]:
main_ing_matrix = main_ing_df

for i in unique_main_ing:
    main_ing_matrix = add_main_ing(main_ing_df, i)

<ipython-input-77-804ffff9ca06>:72: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col1] = empty_bool


In [142]:
main_ing_matrix = main_ing_matrix.set_index('Recipe_ID')

In [192]:
main_ing_matrix.any().all(axis=0)
main_ing_matrix.shape

(2192, 132)

In [178]:
for i in main_ing_matrix.index:
    populate_main_ing_table(main_ing_matrix, i)

In [177]:
def populate_main_ing_table(in_df, recipe_ID):
    main_ingredient = in_df.loc[recipe_ID, "Main_Ingredient"]
    #print(main_ingredient)
    col = in_df.filter(like = main_ingredient).columns
    #print(col)
    for i in range(len(col)):
        if col[i] == 'main_ing_' + main_ingredient:
            #print("True! " + col[i] + " is the main ing!")
            in_df.loc[recipe_ID, 'main_ing_' + main_ingredient] = True

        else:
            pass
    return in_df

In [182]:
main_ing_matrix

,Main_Ingredient,main_ing_Pale 2-Row,main_ing_Dry Malt Extract - Extra Light,main_ing_Castle Malting Pilsen 2RP/2RS,main_ing_Thomas Fawcett Pale Ale Malt (Maris Otter),main_ing_Liquid Malt Extract - Light,main_ing_Pilsner,main_ing_Pale Ale,main_ing_Maris Otter Pale,main_ing_Thomas Fawcett Maris Otter Pale Malt,...,main_ing_Best Pilsen Malt,main_ing_Munich Light - DE,main_ing_Standard 2-Row,"main_ing_Lager Malt, Argyle Craft Malts (NY)",main_ing_GoldSwaen Munich Light,main_ing_German - Pale Ale,main_ing_Joe White Pilsner,main_ing_Pilsener,main_ing_Maris Otter Pale Malt,main_ing_Viking Pale Ale
Recipe_ID,,,,,,,,,,,,,,,,,,,,,
1633,Pale 2-Row,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
16367,Pale 2-Row,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5920,Dry Malt Extract - Extra Light,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5916,Pale 2-Row,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
89534,Castle Malting Pilsen 2RP/2RS,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606909,Pale Ale,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
618500,Maris Otter Pale,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
616365,Pilsner,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [181]:
ing_ferm_table = ing_table_ferm_df.merge(main_ing_matrix, how='inner', on = 'Recipe_ID')
ing_ferm_table

,Recipe_ID,Percent_Adjunct(%),Percent_Dry_Extract(%),Percent_Extract(%),Percent_Grain(%),Percent_Sugar(%),Percent_Ferm_Total(%),Main_Ingredient_x,Pale 2-Row,White Wheat,...,main_ing_Best Pilsen Malt,main_ing_Munich Light - DE,main_ing_Standard 2-Row,"main_ing_Lager Malt, Argyle Craft Malts (NY)",main_ing_GoldSwaen Munich Light,main_ing_German - Pale Ale,main_ing_Joe White Pilsner,main_ing_Pilsener,main_ing_Maris Otter Pale Malt,main_ing_Viking Pale Ale
0,1633,0.063830,0.000000,0.0,0.872340,0.06383,1.0,Pale 2-Row,2.381360,0.907185,...,False,False,False,False,False,False,False,False,False,False
1,16367,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,6.803886,NaN,...,False,False,False,False,False,False,False,False,False,False
2,5920,0.000000,0.705882,0.0,0.294118,0.00000,1.0,Dry Malt Extract - Extra Light,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,5916,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,5.329710,NaN,...,False,False,False,False,False,False,False,False,False,False
4,89534,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Castle Malting Pilsen 2RP/2RS,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,606909,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale Ale,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2188,618500,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Maris Otter Pale,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2189,616365,0.050000,0.000000,0.0,0.950000,0.00000,1.0,Pilsner,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2190,134193,0.021277,0.000000,0.0,0.978723,0.00000,1.0,Pilsner,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [193]:
ing_ferm_table.to_csv('full_ferm_table_with_perc_amt_mi.csv')

In [299]:
#create blank hops ingredients table

# recipe_id column and the percent columns first

h = {'Recipe_ID': 0, 'Percent_Aroma(%)': 0,'Percent_Boil(%)': 0, 'Percent_Dry_Hop(%)': 0,'Percent_First_Wort(%)': 0, 'Percent_Mash(%)': 0,'Percent_Hop_Total(%)': 0}
h_ing_df = pd.DataFrame(h, index= [0])
h_ing_df.head()

,Recipe_ID,Percent_Aroma(%),Percent_Boil(%),Percent_Dry_Hop(%),Percent_First_Wort(%),Percent_Mash(%),Percent_Hop_Total(%)
0,0,0,0,0,0,0,0


In [300]:
for i in hops_df['NAME'].unique():
    
    h_ing_unique_df = add_hops_ing(h_ing_df, i)

<ipython-input-39-51fbec60ac29>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col2] = amt
<ipython-input-39-51fbec60ac29>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col3] = amt
<ipython-input-39-51fbec60ac29>:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col4] = amt
<ipytho

In [301]:
h_ing_unique_df

,Recipe_ID,Percent_Aroma(%),Percent_Boil(%),Percent_Dry_Hop(%),Percent_First_Wort(%),Percent_Mash(%),Percent_Hop_Total(%),Cascade,alpha_Cascade,use_Aroma_Cascade,...,use_DryHop_Mos-cen-dor-sim,use_FirstWort_Mos-cen-dor-sim,use_Mash_Mos-cen-dor-sim,experimental 035,alpha_experimental 035,use_Aroma_experimental 035,use_Boil_experimental 035,use_DryHop_experimental 035,use_FirstWort_experimental 035,use_Mash_experimental 035
0,0,0,0,0,0,0,0,False,0,0,...,0,0,0,False,0,0,0,0,0,0


In [302]:
def populate_hops_table(in_df, recipe_ID):
    column_dict = {'Recipe_ID': 0, 'Percent_Aroma(%)': 0,'Percent_Boil(%)': 0, 'Percent_Dry_Hop(%)': 0,'Percent_First_Wort(%)': 0, 'Percent_Mash(%)': 0,'Percent_Hop_Total(%)': 0}
    out_df = pd.DataFrame(column_dict, index= [recipe_ID])
    
    
    temp_list = add_hop_percents_to_table(recipe_ID)
    
    out_df['Recipe_ID'] = temp_list[0]
    out_df['Percent_Aroma(%)'] = temp_list[1]
    out_df['Percent_Boil(%)'] = temp_list[2]
    out_df['Percent_Dry_Hop(%)'] = temp_list[3]
    out_df['Percent_First_Wort(%)'] = temp_list[4]
    out_df['Percent_Mash(%)'] = temp_list[5]
    out_df['Percent_Hop_Total(%)'] = sum(temp_list[1:6])
    
    temp_recipe = RecipeController.find_recipe_by_id(recipe_ID)
    hop_array = []
    
    for i in temp_recipe.hops:
        hop_array.append(i)
       
    for j in hop_array:
        out_df[j.name] = True
        out_df['alpha_'+j.name] = j.alpha
        #hop_name = verify_hop_name(in_df, j.name)
      
        if j.hop_use == 'Aroma':
            if 'use_Aroma_'+ j.name in out_df.columns:
                out_df['use_Aroma_'+ j.name] = j.amount + out_df.loc[recipe_ID, 'use_Aroma_'+ j.name]
            else:
                out_df['use_Aroma_'+ j.name] = j.amount 
       
        elif j.hop_use == 'Boil':
            if 'use_Boil_'+ j.name in out_df.columns:
                out_df['use_Boil_'+ j.name] = j.amount + out_df.loc[recipe_ID, 'use_Boil_'+ j.name]
            else:
                out_df['use_Boil_'+ j.name] = j.amount 
        
        elif j.hop_use == 'Dry Hop':
            if 'use_DryHop_'+ j.name in out_df.columns:
                out_df['use_DryHop_'+ j.name] = j.amount + out_df.loc[recipe_ID, 'use_DryHop_'+ j.name]
            else:
                out_df['use_DryHop_'+ j.name] = j.amount 
        elif j.hop_use == 'First Wort':
            if 'use_FirstWort_'+ j.name in out_df.columns:
                out_df['use_FirstWort_'+ j.name] = j.amount + out_df.loc[recipe_ID, 'use_FirstWort_'+ j.name]
            else:
                out_df['use_FirstWort_'+ j.name] = j.amount
        else: #j.hop_use == 'Mash'
            if 'use_Mash_'+ j.name in out_df.columns:
                out_df['use_Mash_'+ j.name] = j.amount + out_df.loc[recipe_ID, 'use_Mash_'+ j.name]
            else:
                out_df['use_Mash_'+ j.name] = j.amount
                
    frames = [in_df, out_df]
    result_df = pd.concat(frames, axis=0)
    
    return result_df

def add_hop_percents_to_table(recipe_id):
    percents = RecipeController.find_percent_hop_use_by_id(recipe_id)
    aroma = percents[0]
    boil = percents[1]
    dry = percents[2]
    first = percents[3]
    mash = percents[4]

    data = [recipe_id, aroma, boil, dry, first, mash]
    return data

def verify_hop_name(in_df, hop):
    col = in_df.filter(like = hop).columns
    #print(col)
    for i in range(len(col)):
        if col[i] == hop:
            print("True! " + col[i] + " is the hop name!")
            return col[i]
        else:
            pass
    

In [303]:
a = 0
for i in refined_recipe_df['Recipe ID']:
    h_ing_unique_df = populate_hops_table(h_ing_unique_df, i)

    a += 1
    if a % 100 == 0:
        print("Just made row #" +str(a))

Just made row #100
Just made row #200
Just made row #300
Just made row #400
Just made row #500
Just made row #600
Just made row #700
Just made row #800
Just made row #900
Just made row #1000
Just made row #1100
Just made row #1200
Just made row #1300
Just made row #1400
Just made row #1500
Just made row #1600
Just made row #1700
Just made row #1800
Just made row #1900
Just made row #2000
Just made row #2100


Recipe ID #21783 was a rootbeer recipe and did not have any hops in it. It was removed from the data set as it's outside the scope of this project. https://www.brewersfriend.com/homebrew/recipe/view/21783/zatarain-s-root-beer-non-alcoholic-keg-instructions 

Recipie ID #314560 is not a real recipe as stated by the author. https://www.brewersfriend.com/homebrew/recipe/view/314560/canned-starter

Recipe ID #7657 is a cider. It is brewed but doesn't use hops. https://www.brewersfriend.com/homebrew/recipe/view/7657/mogs-cottage-cider

In [312]:
h_ing_unique_df = h_ing_unique_df.iloc[1:, :]
h_ing_unique_df

,Recipe_ID,Percent_Aroma(%),Percent_Boil(%),Percent_Dry_Hop(%),Percent_First_Wort(%),Percent_Mash(%),Percent_Hop_Total(%),Cascade,alpha_Cascade,use_Aroma_Cascade,...,use_DryHop_Mos-cen-dor-sim,use_FirstWort_Mos-cen-dor-sim,use_Mash_Mos-cen-dor-sim,experimental 035,alpha_experimental 035,use_Aroma_experimental 035,use_Boil_experimental 035,use_DryHop_experimental 035,use_FirstWort_experimental 035,use_Mash_experimental 035
16367,16367,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5920,5920,0.000000,0.500000,0.375000,0.125000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5916,5916,0.000000,0.571429,0.342857,0.085714,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89534,89534,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28546,28546,0.000000,0.666667,0.333333,0.000000,0.0,1.0,True,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606909,606909,0.000000,0.754300,0.245700,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618500,618500,0.000000,0.571198,0.342880,0.085922,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616365,616365,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134193,134193,0.000000,0.833333,0.166667,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [313]:
h_ing_unique_df.isnull().all(axis=0)

Recipe_ID                         False
Percent_Aroma(%)                  False
Percent_Boil(%)                   False
Percent_Dry_Hop(%)                False
Percent_First_Wort(%)             False
                                  ...  
use_Aroma_experimental 035         True
use_Boil_experimental 035          True
use_DryHop_experimental 035        True
use_FirstWort_experimental 035     True
use_Mash_experimental 035          True
Length: 22883, dtype: bool

In [314]:
h_ing_unique_df = h_ing_unique_df.dropna(axis=1, how='all')
h_ing_unique_df.shape

(2189, 1392)

In [315]:
h_ing_unique_df

,Recipe_ID,Percent_Aroma(%),Percent_Boil(%),Percent_Dry_Hop(%),Percent_First_Wort(%),Percent_Mash(%),Percent_Hop_Total(%),Cascade,alpha_Cascade,use_Aroma_Cascade,...,Simcoe DBL,alpha_Simcoe DBL,use_Boil_Simcoe DBL,EXPERIMENTAL GRAPE,alpha_EXPERIMENTAL GRAPE,use_Aroma_EXPERIMENTAL GRAPE,use_DryHop_EXPERIMENTAL GRAPE,Columbus/Tomahawk,alpha_Columbus/Tomahawk,use_Boil_Columbus/Tomahawk
16367,16367,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5920,5920,0.000000,0.500000,0.375000,0.125000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5916,5916,0.000000,0.571429,0.342857,0.085714,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89534,89534,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28546,28546,0.000000,0.666667,0.333333,0.000000,0.0,1.0,True,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606909,606909,0.000000,0.754300,0.245700,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618500,618500,0.000000,0.571198,0.342880,0.085922,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616365,616365,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134193,134193,0.000000,0.833333,0.166667,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [316]:
h_ing_unique_df.to_csv('ing_table_hops.csv')

YEAST

In [74]:
#create blank ferm ingredients table

# recipe_id column and the percent Ferm_type% columns first

y = {'Recipe_ID': 0, 'Floc_Low': 0,'Floc_Med': 0, 'Floc_High': 0,'Floc_VeryHigh': 0, 'Form_Dry': 0,'Form_Liquid': 0, 'Form_Slurry': 0}
y_ing_df = pd.DataFrame(y, index= [0])
y_ing_df.head()

,Recipe_ID,Floc_Low,Floc_Med,Floc_High,Floc_VeryHigh,Form_Dry,Form_Liquid,Form_Slurry
0,0,0,0,0,0,0,0,0


In [75]:
y_ing_df

,Recipe_ID,Floc_Low,Floc_Med,Floc_High,Floc_VeryHigh,Form_Dry,Form_Liquid,Form_Slurry
0,0,0,0,0,0,0,0,0


In [76]:
for i in yeast_df['NAME'].unique():
    
    y_ing_name_df = add_yeast_name(y_ing_df, i)

for j in yeast_df['LABORATORY'].unique():
    y_ing_name_lab_df = add_yeast_lab(y_ing_name_df, j)
  
for k in yeast_df['TYPE'].unique():
    y_ing_name_lab_type_df = add_yeast_type(y_ing_name_lab_df, k)

<ipython-input-39-51fbec60ac29>:39: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col1]= empty_bool
<ipython-input-39-51fbec60ac29>:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col1] = empty_bool
<ipython-input-39-51fbec60ac29>:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col1] 

In [77]:
y_ing_name_lab_type_df.head()

,Recipe_ID,Floc_Low,Floc_Med,Floc_High,Floc_VeryHigh,Form_Dry,Form_Liquid,Form_Slurry,Kölsch 2565,American Ale 1056,...,ytype_hybrid,ytype_Bacterial Cultures,ytype_Brettanomyces,ytype_BRETTANOMYCES,ytype_nan,ytype_ale,ytype_Bretts and Blends,ytype_German Ales,ytype_S. boulardii,ytype_Mead
0,0,0,0,0,0,0,0,0,False,False,...,False,False,False,False,False,False,False,False,False,False


In [78]:
def populate_yeast_table(in_df, recipe_ID):
    
    '''use the recipe_id to populate the yeast table boolean values'''
    
    recipe = RecipeController.find_recipe_by_id(recipe_ID)
    yeast = recipe.yeast
    name = str(yeast.name)
    lab = str(yeast.laboratory)
    floc = str(yeast.flocculation)
    form = str(yeast.form)
    ytype = str(yeast.yeast_type)
  

    '''populate name'''
    col = in_df.filter(like = name).columns
    
    for i in range(len(col)):
        if col[i] == name:
            in_df.loc[recipe_ID, name] = True
        else:
            pass

    '''populate lab'''
    col = in_df.filter(like = lab).columns
    
    for i in range(len(col)):
        if col[i] == 'lab_' + lab:
            in_df.loc[recipe_ID, 'lab_' + lab] = True
        else:
            pass
    
    '''populate yeast form'''
    col = in_df.filter(like = form).columns
    
    for i in range(len(col)):
        if col[i] == 'Form_' + form:
            in_df.loc[recipe_ID, 'Form_' + form] = True
        else:
            pass
    
    '''populate floc'''
    col = in_df.filter(like = floc).columns
    
    for i in range(len(col)):
        if col[i] == 'Floc_' + floc:
            in_df.loc[recipe_ID, 'Floc_' + floc] = True
        else:
            pass
    
    '''populate ytype'''
    col = in_df.filter(like = ytype).columns
    
    for i in range(len(col)):
        if col[i] == 'ytype_' + ytype:
            in_df.loc[recipe_ID, 'ytype_' + ytype] = True
        else:
            pass    
    
    return in_df

In [79]:
a = 0
for i in refined_recipe_df['Recipe ID']:
    y_ing_name_lab_type_df = populate_yeast_table(y_ing_name_lab_type_df, i)

    a += 1
    if a % 100 == 0:
        print("Just made row #" +str(a))

Just made row #100
Just made row #200
Just made row #300
Just made row #400
Just made row #500
Just made row #600
Just made row #700
Just made row #800
Just made row #900
Just made row #1000
Just made row #1100
Just made row #1200
Just made row #1300
Just made row #1400
Just made row #1500
Just made row #1600
Just made row #1700
Just made row #1800
Just made row #1900
Just made row #2000
Just made row #2100


In [60]:
y_ing_name_lab_type_df.head()

,Recipe_ID,Floc_Low,Floc_Med,Floc_High,Floc_VeryHigh,Form_Dry,Form_Liquid,Form_Slurry,Kölsch 2565,American Ale 1056,...,ytype_hybrid,ytype_Bacterial Cultures,ytype_Brettanomyces,ytype_BRETTANOMYCES,ytype_nan,ytype_ale,ytype_Bretts and Blends,ytype_German Ales,ytype_S. boulardii,ytype_Mead
0,620065.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,...,False,False,False,False,False,False,False,False,False,False
1633,1633.0,True,1633.0,1633,1633.0,1633,True,1633.0,True,1633,...,1633,1633,1633,1633,1633,1633,1633,1633,1633,1633
16367,16367.0,16367,16367.0,16367,16367.0,16367,True,16367.0,16367,True,...,16367,16367,16367,16367,16367,16367,16367,16367,16367,16367
5920,5920.0,5920,5920.0,True,5920.0,True,5920,5920.0,5920,5920,...,5920,5920,5920,5920,5920,5920,5920,5920,5920,5920
5916,5916.0,5916,5916.0,True,5916.0,True,5916,5916.0,5916,5916,...,5916,5916,5916,5916,5916,5916,5916,5916,5916,5916


In [82]:
y_ing_name_lab_type_df = y_ing_name_lab_type_df.iloc[1:,:]

In [85]:
for i in refined_recipe_df['Recipe ID']:
    y_ing_name_lab_type_df.loc[i, 'Recipe_ID'] = i


y_ing_name_lab_type_df

C:\Users\Andrew\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Recipe_ID,Floc_Low,Floc_Med,Floc_High,Floc_VeryHigh,Form_Dry,Form_Liquid,Form_Slurry,Kölsch 2565,American Ale 1056,...,ytype_hybrid,ytype_Bacterial Cultures,ytype_Brettanomyces,ytype_BRETTANOMYCES,ytype_nan,ytype_ale,ytype_Bretts and Blends,ytype_German Ales,ytype_S. boulardii,ytype_Mead
1633,1633.0,True,NaN,NaN,NaN,NaN,True,NaN,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16367,16367.0,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5920,5920.0,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5916,5916.0,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89534,89534.0,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606909,606909.0,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618500,618500.0,NaN,NaN,True,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616365,616365.0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134193,134193.0,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
y_ing_name_lab_type_df.shape

(2192, 1790)

In [87]:
y_ing_name_lab_type_df.isnull().all(axis=0)

Recipe_ID                  False
Floc_Low                   False
Floc_Med                    True
Floc_High                  False
Floc_VeryHigh               True
                           ...  
ytype_ale                   True
ytype_Bretts and Blends    False
ytype_German Ales           True
ytype_S. boulardii          True
ytype_Mead                  True
Length: 1790, dtype: bool

In [88]:
y_ing_name_lab_type_df = y_ing_name_lab_type_df.dropna(axis=1, how='all')
y_ing_name_lab_type_df.shape

(2192, 239)

In [89]:
y_ing_name_lab_type_df

,Recipe_ID,Floc_Low,Floc_High,Form_Dry,Form_Liquid,Kölsch 2565,American Ale 1056,Safale - English Ale Yeast S-04,Safbrew - Specialty Ale Yeast T-58,Safale - American Ale Yeast US-05,...,ytype_Lagers,ytype_Norwegian Ales,ytype_Wheat,ytype_Ales,ytype_Wine,ytype_British Ales,ytype_Cider,ytype_Belgian Ale,ytype_German Ale,ytype_Bretts and Blends
1633,1633.0,True,NaN,NaN,True,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16367,16367.0,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5920,5920.0,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5916,5916.0,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89534,89534.0,NaN,True,True,NaN,NaN,NaN,NaN,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606909,606909.0,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
618500,618500.0,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
616365,616365.0,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134193,134193.0,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
y_ing_name_lab_type_df.to_csv('ing_table_yeast.csv')

merging the tables together

In [22]:
ing_ferm_table_df = pd.read_csv('full_ferm_table_with_perc_amt_mi.csv')
ing_yeast_table_df = pd.read_csv('ing_table_yeast.csv')

In [23]:
ing_ferm_table_df = ing_ferm_table_df.drop('Unnamed: 0', axis=1)
ing_ferm_table_df

,Recipe_ID,Percent_Adjunct(%),Percent_Dry_Extract(%),Percent_Extract(%),Percent_Grain(%),Percent_Sugar(%),Percent_Ferm_Total(%),Main_Ingredient_x,Pale 2-Row,White Wheat,...,main_ing_Best Pilsen Malt,main_ing_Munich Light - DE,main_ing_Standard 2-Row,"main_ing_Lager Malt, Argyle Craft Malts (NY)",main_ing_GoldSwaen Munich Light,main_ing_German - Pale Ale,main_ing_Joe White Pilsner,main_ing_Pilsener,main_ing_Maris Otter Pale Malt,main_ing_Viking Pale Ale
0,1633,0.063830,0.000000,0.0,0.872340,0.06383,1.0,Pale 2-Row,2.381360,0.907185,...,False,False,False,False,False,False,False,False,False,False
1,16367,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,6.803886,NaN,...,False,False,False,False,False,False,False,False,False,False
2,5920,0.000000,0.705882,0.0,0.294118,0.00000,1.0,Dry Malt Extract - Extra Light,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
3,5916,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale 2-Row,5.329710,NaN,...,False,False,False,False,False,False,False,False,False,False
4,89534,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Castle Malting Pilsen 2RP/2RS,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,606909,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Pale Ale,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2188,618500,0.000000,0.000000,0.0,1.000000,0.00000,1.0,Maris Otter Pale,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2189,616365,0.050000,0.000000,0.0,0.950000,0.00000,1.0,Pilsner,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
2190,134193,0.021277,0.000000,0.0,0.978723,0.00000,1.0,Pilsner,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False


In [24]:
ing_hop_table_df = pd.read_csv('ing_table_hops.csv')
ing_hop_table_df = ing_hop_table_df.drop('Unnamed: 0', axis=1)

C:\Users\Andrew\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (15,24,68,83,151,179,191,195,200,234,239,242,245,248,258,264,267,276,281,291,312,315,318,321,336,341,344,347,350,356,360,365,370,374,378,382,386,392,396,399,402,405,408,413,419,428,431,434,437,441,444,447,451,455,460,463,468,471,480,483,496,500,503,506,509,514,517,520,525,528,532,543,546,549,552,556,563,569,572,576,581,586,589,592,598,603,607,612,616,621,624,627,631,635,638,641,645,648,653,656,659,662,666,669,672,675,678,683,687,690,693,696,702,706,711,714,719,724,727,730,733,737,742,746,750,753,756,760,763,768,771,775,779,782,788,791,794,797,801,804,807,813,816,819,822,826,829,833,838,841,844,847,850,855,858,862,865,868,871,874,879,883,888,892,895,899,902,905,908,911,914,917,920,924,929,934,937,941,945,948,951,954,957,961,964,967,970,974,978,981,985,988,993,996,999,1002,1005,1009,1013,1017,1020,1023,1026,1029,1032,1036,1039,1042,1045,1048,1051,1054,1057,1060,1063,1066,1069,1073,107

In [25]:
ing_hop_table_df

,Recipe_ID,Percent_Aroma(%),Percent_Boil(%),Percent_Dry_Hop(%),Percent_First_Wort(%),Percent_Mash(%),Percent_Hop_Total(%),Cascade,alpha_Cascade,use_Aroma_Cascade,...,Simcoe DBL,alpha_Simcoe DBL,use_Boil_Simcoe DBL,EXPERIMENTAL GRAPE,alpha_EXPERIMENTAL GRAPE,use_Aroma_EXPERIMENTAL GRAPE,use_DryHop_EXPERIMENTAL GRAPE,Columbus/Tomahawk,alpha_Columbus/Tomahawk,use_Boil_Columbus/Tomahawk
0,16367,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5920,0.000000,0.500000,0.375000,0.125000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5916,0.000000,0.571429,0.342857,0.085714,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89534,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28546,0.000000,0.666667,0.333333,0.000000,0.0,1.0,True,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,606909,0.000000,0.754300,0.245700,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2185,618500,0.000000,0.571198,0.342880,0.085922,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2186,616365,0.000000,1.000000,0.000000,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2187,134193,0.000000,0.833333,0.166667,0.000000,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
ing_yeast_table_df = ing_yeast_table_df.drop('Unnamed: 0', axis=1)
ing_yeast_table_df

,Recipe_ID,Floc_Low,Floc_High,Form_Dry,Form_Liquid,Kölsch 2565,American Ale 1056,Safale - English Ale Yeast S-04,Safbrew - Specialty Ale Yeast T-58,Safale - American Ale Yeast US-05,...,ytype_Lagers,ytype_Norwegian Ales,ytype_Wheat,ytype_Ales,ytype_Wine,ytype_British Ales,ytype_Cider,ytype_Belgian Ale,ytype_German Ale,ytype_Bretts and Blends
0,1633.0,True,NaN,NaN,True,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16367.0,NaN,NaN,NaN,True,NaN,True,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5920.0,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5916.0,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,89534.0,NaN,True,True,NaN,NaN,NaN,NaN,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,606909.0,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2188,618500.0,NaN,True,True,NaN,NaN,NaN,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2189,616365.0,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2190,134193.0,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,True,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
ing_tables = [ing_ferm_table_df, ing_hop_table_df, ing_yeast_table_df]
full_data = refined_recipe_df

In [28]:
ferm_hop_table = ing_ferm_table_df.merge(ing_hop_table_df, on = 'Recipe_ID')
ing_table = ferm_hop_table.merge(ing_yeast_table_df, on = "Recipe_ID")
ing_table

,Recipe_ID,Percent_Adjunct(%),Percent_Dry_Extract(%),Percent_Extract(%),Percent_Grain(%),Percent_Sugar(%),Percent_Ferm_Total(%),Main_Ingredient_x,Pale 2-Row,White Wheat,...,ytype_Lagers,ytype_Norwegian Ales,ytype_Wheat,ytype_Ales,ytype_Wine,ytype_British Ales,ytype_Cider,ytype_Belgian Ale,ytype_German Ale,ytype_Bretts and Blends
0,16367,0.000000,0.000000,0.0,1.000000,0.0,1.0,Pale 2-Row,6.803886,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5920,0.000000,0.705882,0.0,0.294118,0.0,1.0,Dry Malt Extract - Extra Light,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5916,0.000000,0.000000,0.0,1.000000,0.0,1.0,Pale 2-Row,5.329710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89534,0.000000,0.000000,0.0,1.000000,0.0,1.0,Castle Malting Pilsen 2RP/2RS,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28546,0.000000,0.000000,0.0,1.000000,0.0,1.0,Pale 2-Row,5.216312,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2184,606909,0.000000,0.000000,0.0,1.000000,0.0,1.0,Pale Ale,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2185,618500,0.000000,0.000000,0.0,1.000000,0.0,1.0,Maris Otter Pale,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2186,616365,0.050000,0.000000,0.0,0.950000,0.0,1.0,Pilsner,NaN,NaN,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2187,134193,0.021277,0.000000,0.0,0.978723,0.0,1.0,Pilsner,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
full_data_table = refined_recipe_df.merge(ing_table, how='left', left_on = "Recipe ID", right_on = 'Recipe_ID')
full_data_table

,Unnamed: 0,BeerID,Name,URL,Style,StyleID,Size(L),OG,FG,ABV,...,ytype_Lagers,ytype_Norwegian Ales,ytype_Wheat,ytype_Ales,ytype_Wine,ytype_British Ales,ytype_Cider,ytype_Belgian Ale,ytype_German Ale,ytype_Bretts and Blends
0,0,1,Vanilla Cream Ale,/homebrew/recipe/view/1633/vanilla-cream-ale,Cream Ale,45,21.77,1.055,1.013,5.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,Southern Tier Pumking clone,/homebrew/recipe/view/16367/southern-tier-pumk...,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,Zombie Dust Clone - EXTRACT,/homebrew/recipe/view/5920/zombie-dust-clone-e...,American IPA,7,18.93,1.063,1.018,5.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/5916/zombie-dust-clone-a...,American IPA,7,22.71,1.061,1.017,5.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,Bakke Brygg Belgisk Blonde 50 l,/homebrew/recipe/view/89534/bakke-brygg-belgis...,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,73375,73376,Käftsmälls IPA,/homebrew/recipe/view/606909/,American IPA,7,10.00,1.064,1.012,6.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2186,73432,73433,Zombie Dust Clone - ALL GRAIN,/homebrew/recipe/view/618500/zombie-dust-clone...,American IPA,7,22.70,1.061,1.014,6.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2187,73549,73550,German pils,/homebrew/recipe/view/616365/german-pils,German Pils,80,27.00,1.050,1.011,5.06,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2188,73628,73629,Magelan IPA 2018.,/homebrew/recipe/view/134193/magelan-ipa-2018-,English IPA,65,30.00,1.060,1.011,6.39,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
full_data_table = full_data_table.drop(['Unnamed: 0', 'BeerID', 'URL'], axis=1)


In [35]:
full_data_table

,Name,Style,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,...,ytype_Lagers,ytype_Norwegian Ales,ytype_Wheat,ytype_Ales,ytype_Wine,ytype_British Ales,ytype_Cider,ytype_Belgian Ale,ytype_German Ale,ytype_Bretts and Blends
0,Vanilla Cream Ale,Cream Ale,45,21.77,1.055,1.013,5.48,17.65,4.83,28.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Southern Tier Pumking clone,Holiday/Winter Special Spiced Beer,85,20.82,1.083,1.021,8.16,60.65,15.64,24.61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Zombie Dust Clone - EXTRACT,American IPA,7,18.93,1.063,1.018,5.91,59.25,8.98,22.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Zombie Dust Clone - ALL GRAIN,American IPA,7,22.71,1.061,1.017,5.80,54.48,8.50,26.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Bakke Brygg Belgisk Blonde 50 l,Belgian Blond Ale,20,50.00,1.060,1.010,6.48,17.84,4.57,60.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,Käftsmälls IPA,American IPA,7,10.00,1.064,1.012,6.85,73.84,4.73,12.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2186,Zombie Dust Clone - ALL GRAIN,American IPA,7,22.70,1.061,1.014,6.13,62.79,7.57,26.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2187,German pils,German Pils,80,27.00,1.050,1.011,5.06,27.84,2.95,30.00,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2188,Magelan IPA 2018.,English IPA,65,30.00,1.060,1.011,6.39,59.12,11.62,45.00,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
full_data_table.to_csv('full_data_table.csv')